# Grossmünster #311/918, scene 0

In [6]:
!s5cmd --no-sign-request ls s3://megascenes/reconstruct/311/918/colmap/0/

2024/05/30 08:27:30             21848  cameras.bin
2024/05/30 08:27:30          73827844  images.bin
2024/05/30 08:27:30           7057803  points3D.bin
2024/05/30 08:27:30              1748  project.ini


In [7]:
!s5cmd --no-sign-request cp s3://megascenes/reconstruct/311/918/colmap/0/* ./reconstruct/311/918/colmap/0/
!ls -lah ./reconstruct/311/918/colmap/0/

cp s3://megascenes/reconstruct/311/918/colmap/0/cameras.bin reconstruct/311/918/colmap/0/cameras.bin
cp s3://megascenes/reconstruct/311/918/colmap/0/project.ini reconstruct/311/918/colmap/0/project.ini
cp s3://megascenes/reconstruct/311/918/colmap/0/points3D.bin reconstruct/311/918/colmap/0/points3D.bin
cp s3://megascenes/reconstruct/311/918/colmap/0/images.bin reconstruct/311/918/colmap/0/images.bin
total 78M
drwxr-xr-x 1 vscode vscode 4.0K Nov  7 16:30 .
drwxr-xr-x 1 vscode vscode 4.0K Nov  7 16:25 ..
-rw-r--r-- 1 vscode vscode  22K Nov  7 16:30 cameras.bin
-rw-r--r-- 1 vscode vscode  71M Nov  7 16:30 images.bin
-rw-r--r-- 1 vscode vscode 6.8M Nov  7 16:30 points3D.bin
-rw-r--r-- 1 vscode vscode 1.8K Nov  7 16:30 project.ini


In [1]:
import pycolmap
reconstruction = pycolmap.Reconstruction("./reconstruct/311/918/colmap/0/")
reconstruction.summary()

'Reconstruction:\n\tnum_reg_images = 540\n\tnum_cameras = 390\n\tnum_points3D = 65737\n\tnum_observations = 463151\n\tmean_track_length = 7.04551\n\tmean_observations_per_image = 857.687\n\tmean_reprojection_error = 0.926982'

In [2]:
for image_id, image in reconstruction.images.items():
    print(image_id, image)

447 Image(image_id=447, camera_id=355, name="commons/Grossmünster/0/pictures/Grossmünster_2012.jpg", triangulated=1291/8791)
435 Image(image_id=435, camera_id=358, name="commons/Grossmünster/0/pictures/Grossmünster_Zürich_-_1.jpg", triangulated=612/4043)
424 Image(image_id=424, camera_id=340, name="commons/Grossmünster/0/pictures/Grossmünster_-_Limmatquai_-_ZSG_Limmatschif_'Regula'_2013-06-16_18-22-18_(P7700).JPG", triangulated=354/7984)
965 Image(image_id=965, camera_id=664, name="commons/Night_views_of_Grossmünster/0/pictures/Grossmunster_Zurich_Ank_Kumar_08.jpg", triangulated=2379/5451)
417 Image(image_id=417, camera_id=242, name="commons/Grossmünster/0/pictures/Grossmünster_-_Lindenhof_2011-04-07_19-16-26_ShiftN.jpg", triangulated=879/3178)
413 Image(image_id=413, camera_id=332, name="commons/Grossmünster/0/pictures/Grossmünster_-_Helmhaus-Wasserkirche_-_Münsterbrücke_-_Münsterhof_2011-08-01_16-20-44_ShiftN.jpg", triangulated=596/4382)
400 Image(image_id=400, camera_id=305, name="c

In [5]:
for camera_id, camera in reconstruction.cameras.items():
    print(camera_id, camera)

423 Camera(camera_id=423, model=SIMPLE_RADIAL, width=1800, height=1217, params=[3125.279680, 900.000000, 608.500000, -0.278064] (f, cx, cy, k))
421 Camera(camera_id=421, model=SIMPLE_RADIAL, width=1800, height=2700, params=[5388.090796, 900.000000, 1350.000000, -0.649623] (f, cx, cy, k))
417 Camera(camera_id=417, model=SIMPLE_RADIAL, width=1800, height=2400, params=[5482.988939, 900.000000, 1200.000000, -0.298410] (f, cx, cy, k))
413 Camera(camera_id=413, model=SIMPLE_RADIAL, width=500, height=331, params=[537.058898, 250.000000, 165.500000, -0.236472] (f, cx, cy, k))
411 Camera(camera_id=411, model=SIMPLE_RADIAL, width=295, height=450, params=[717.916342, 147.500000, 225.000000, -0.197528] (f, cx, cy, k))
403 Camera(camera_id=403, model=SIMPLE_RADIAL, width=1600, height=1200, params=[553.623790, 800.000000, 600.000000, 0.010137] (f, cx, cy, k))
396 Camera(camera_id=396, model=SIMPLE_RADIAL, width=1800, height=1350, params=[1385.955596, 900.000000, 675.000000, 0.029109] (f, cx, cy, k))

In [8]:
reconstruction.cameras["423"]

TypeError: __getitem__(): incompatible function arguments. The following argument types are supported:
    1. (self: pycolmap.MapCameraIdToCamera, arg0: int) -> pycolmap.Camera

Invoked with: {423: Camera(camera_id=423, model=SIMPLE_RADIAL, width=1800, height=1217, params=[3125.279680, 900.000000, 608.500000, -0.278064] (f, cx, cy, k)),
 421: Camera(camera_id=421, model=SIMPLE_RADIAL, width=1800, height=2700, params=[5388.090796, 900.000000, 1350.000000, -0.649623] (f, cx, cy, k)),
 417: Camera(camera_id=417, model=SIMPLE_RADIAL, width=1800, height=2400, params=[5482.988939, 900.000000, 1200.000000, -0.298410] (f, cx, cy, k)),
 413: Camera(camera_id=413, model=SIMPLE_RADIAL, width=500, height=331, params=[537.058898, 250.000000, 165.500000, -0.236472] (f, cx, cy, k)),
 411: Camera(camera_id=411, model=SIMPLE_RADIAL, width=295, height=450, params=[717.916342, 147.500000, 225.000000, -0.197528] (f, cx, cy, k)),
 403: Camera(camera_id=403, model=SIMPLE_RADIAL, width=1600, height=1200, par